<a href="https://colab.research.google.com/github/kevinmcmanus/cas-tau/blob/master/AstropyTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Astropy

Astropy is a collecction of Python libraries for use in Astronomy. Its capabilities are vast.  See these two links for an overview and to get an idea of the capabilities Astropy provides:

[Learn Astropy](http://learn.astropy.org/)

[Astropy Tutorials](http://learn.astropy.org/tutorials.html)

In this tutorial, we'll just scratch surface to give you a hands-on encounter with Astropy. After this tutorial you'll know where to look for more information.

## Fancy Plots

### Bunch of Set Up Stuff

In [ ]:
!pip install astroquery

In [2]:
# libraries for general data mashing and plotting
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import matplotlib.colors as colors
%matplotlib inline

In [3]:
from astropy.coordinates import Angle, Distance
from astropy.units import Quantity
import astropy.units as u
from astroquery.simbad import Simbad
import astropy.coordinates as coord
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.table import QTable, Table, vstack
from astropy.units import Quantity

### Construct HR-Diagram for the Pleiades with Gaia Data

A first approximation to obtaining the members of the Pleiades open cluster is to execute a cone search of the Gaia Archive using astrometrics from Simbad.
In the case fo the Pleiades, these are:
```
ra = 56.75 degree
dec = 24.1167 degree
parallax = 7.364 mas
pmra = 19.997 mas/year
pmdec = -45.548 mas/year
```

In [ ]:
from astroquery.gaia import Gaia

# construct ADL query using astrometrics from above, add some obs quality constraints
job3 = Gaia.launch_job("SELECT source_id, ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, parallax \
    FROM gaiadr2.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',56.75,24.1167,2))=1 \
    AND abs(pmra_error/pmra)<0.10 \
    AND abs(pmdec_error/pmdec)<0.10 \
    AND pmra IS NOT NULL AND abs(pmra)>0 \
    AND pmdec IS NOT NULL AND abs(pmdec)>0 \
    AND pmra BETWEEN 15 AND 25 \
    AND pmdec BETWEEN -55 AND -40;")
pleiades_table = job3.get_results()

#examine first 10 records
pleiades_table[0:10]

In [5]:
#from astropy.coordinates import Distance
#from astropy.units import Quantity
# get distance from parallax
distance = Distance(parallax=Quantity(pleiades_table['parallax']))

#absolute magnitude is the apparent magnitude (phot_g_mean_mag) scaled up to a standard distance of 10pc.  distmod does this
abs_mag=pleiades_table['phot_g_mean_mag'] - distance.distmod
star_color = pleiades_table['phot_bp_mean_mag'] - pleiades_table['phot_rp_mean_mag']

In [ ]:
#ready to plot!
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(star_color, abs_mag, s=4)
ax.invert_yaxis()

ax.set_title('Color-Magnitude Diagram: Pleiades')
ax.set_xlabel(r'Color: $BP\ - \ RP$')
ax.set_ylabel(r'Absolute Magnitude: $m_g - distmod$')

### Galactic Hydrogen Plot

#### Fetch the Hydrogen Data

Data from the [HI4PI: A full-sky HI survey based on EBHIS and GASS](https://arxiv.org/abs/1610.06175)

Data File format:
```
Byte-by-byte Description of file: nhi_hpx.dat
--------------------------------------------------------------------------------
   Bytes Format  Units   Label     Explanations
--------------------------------------------------------------------------------
   1-  8  I8     ---     HPX       HPX index (HPXINDEX)
  10- 18  F9.5   deg     RAdeg     Right ascension (J2000)
  20- 28  F9.5   deg     DEdeg     Declination (J2000)
  30- 38  F9.5   deg     GLON      Galactic longitude (J2000)
  40- 48  F9.5   deg     GLAT      Galactic latitude (J2000)
  50- 71  E22.15 cm-2    NHI       HI column density
--------------------------------------------------------------------------------
```

In [ ]:
url = 'https://cdsarc.unistra.fr/ftp/J/A+A/594/A116/nhi_hpx.dat.gz'
cols = ['HPX', 'RAdeg','DEdeg','GLON','GLAT', 'NHI']

hi4pi = pd.read_csv(url, delim_whitespace=True, names=cols, header=None)
print(f'hi4pi has {len(hi4pi)} records')


In [ ]:
hi4pi.head()

In [ ]:
#don't run this code -- takes 20 minutes
#fig = plt.figure(figsize=(12,12))
#ax = fig.add_subplot(111)
#ax.scatter(hi4pi.RAdeg, hi4pi.DEdeg, c=hi4pi.NHI, cmap='gnuplot')
#ax.invert_xaxis()

#### Crunch the Data Array

In [ ]:
#round to 1/10 degree precision
hi4pi['RAdeg1'] = np.round(hi4pi.RAdeg,1)
hi4pi['DEdeg1'] = np.round(hi4pi.DEdeg, 1)
hi4pi['GLAT1'] = np.round(hi4pi.GLAT,1)
hi4pi['GLON1'] = np.round(hi4pi.GLON,1)

#get the mean NHI value at the new precision level, rename the columns to good ol' ra and dec

#equatorial coords
hi4pi_eq = hi4pi[['RAdeg1', 'DEdeg1', 'NHI']].groupby(['RAdeg1','DEdeg1']).mean().reset_index()
hi4pi_eq.rename(columns={"RAdeg1":"ra", "DEdeg1":"dec"}, inplace=True)

#galactic coords
hi4pi_gal = hi4pi[['GLON1', 'GLAT1', 'NHI']].groupby(['GLON1','GLAT1']).mean().reset_index()
hi4pi_gal.rename(columns={"GLON1":"l", "GLAT1":"b"}, inplace=True)

#convert everybody to angles and wrap the longitude
hi4pi_eq.ra = Angle(np.array(hi4pi_eq.ra)*u.degree).wrap_at(180*u.degree)
hi4pi_eq.dec = Angle(np.array(hi4pi_eq.dec)*u.degree)

hi4pi_gal.l = Angle(np.array(hi4pi_gal.l)*u.degree).wrap_at(180*u.degree)
hi4pi_gal.b = Angle(np.array(hi4pi_gal.b)*u.degree)

#create the image arrays
#note declination and lattitude in 'x' (row) positions and ra and longitude in 'y' (column) positions
image_eq = pd.pivot_table(hi4pi_eq,index='dec',columns='ra', values='NHI',aggfunc=np.mean).to_numpy( copy=True)

image_gal = pd.pivot_table(hi4pi_gal,index='b',columns='l', values='NHI',aggfunc=np.mean).to_numpy( copy=True)
image_eq.shape, image_gal.shape


In [10]:
# create x and y axis vectors for eq. and gal. coords
ra_vec = Angle(np.linspace(hi4pi_eq.ra.min(), hi4pi_eq.ra.max(), image_eq.shape[1], endpoint=False)*u.degree)
dec_vec= Angle(np.linspace(hi4pi_eq.dec.min(), hi4pi_eq.dec.max(), image_eq.shape[0], endpoint=False)*u.degree)

l_vec = Angle(np.linspace(hi4pi_gal.l.min(), hi4pi_gal.l.max(), image_gal.shape[1], endpoint=False)*u.degree)
b_vec = Angle(np.linspace(hi4pi_gal.b.min(), hi4pi_gal.b.max(), image_gal.shape[0], endpoint=False)*u.degree)

#### Create Plots

In [ ]:
# plot in equatorial coordinates

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111, projection='mollweide')
pcm = ax.pcolormesh(ra_vec.radian,
           dec_vec.radian,
           image_eq,
           cmap='viridis', norm=colors.LogNorm())
ax.grid()
ax.set_title('Neutral Hydrogen Density\nEquatorial Coordinates')

plt.colorbar(pcm, orientation='horizontal',label='Column Density')


In [ ]:
fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111, projection='mollweide')
pcm=ax.pcolormesh(l_vec.radian,
           b_vec.radian,
           image_gal,
           cmap='viridis', norm=colors.LogNorm())
ax.grid()
ax.set_title('Neutral Hydrogen Density\nGalactic Coordinates')

plt.colorbar(pcm, orientation='horizontal',label='Column Density')


## Basics

To install Astropy on your local machine, execute the following:
```
pip install astropy
```
or if you're using Anaconda to manage your Python environments
```
conda install astropy
```

In the environment provided by Google Colab, astropy is already provided so we don't need to do anything.

The next few sub-sections demonstrate some operations with Constants, Units and Coordinates that the more advanced Astropy modules rely on.

### Constants

Many (most) astronomical calculations involve constants: speed of light, graviational constant, Stefan-Boltzman constant and the like.  Astropy provides many of these, in a module called (what else) `constants`

For a more in-depth discussion of Astropy contants, see: 
[Constants Tutorial](https://docs.astropy.org/en/stable/constants/index.html)

In [13]:
# need the constants module
import astropy.constants as const

In [ ]:
# Gravitational constant G
const.G

In [ ]:
print(const.G)

In [ ]:
#Planck's constant
print (const.h)

In [ ]:
#speed of light
const.c

In [ ]:
#want G in cgs?
const.G.cgs

See [List of Constants](https://docs.astropy.org/en/stable/constants/) for complete list of available constants.

### Units

Units allow us to attach dimensions to scalar and vector quantities. Doing so creates an Astropy `Quantity` object, which among other things allows us to convert among units with relative ease.

In [19]:
#load up the units module
import astropy.units as u

In [ ]:
u.watt

In [ ]:
632.3 * u.watt

In [ ]:
type(632.3), type(u.watt), type(632.3*u.watt)

In [23]:
w = 632.3*u.watt

In [ ]:
#what did we get, how to take it apart
type(w), w.value, w.unit

### Unit Conversion

We can convert from one set of units to another, say nanometers to Angstroms using the Quantity `to` method, below for the Hydrogen Alpha line at 656.28 nanometers:

In [25]:
# specify the h_alpha line in nanometers
h_alpha =  656.28*u.nm

In [ ]:
#convert to angrstrom:
h_alpha.to(u.angstrom)

Divide the h_alpha wavelength: ($\lambda$) into the speed of light to get frequency:
nu = const.c/h_alpha

In [ ]:
nu = const.c/h_alpha
nu

Hmmm, meters per nanometers per second, technically correct, but not very useful. Let's look at this in gigahertz

In [ ]:
nu.to(u.gigahertz)

In [29]:
#work with angles:
from astropy.coordinates import Angle

In [30]:
def showdeg(theta:Angle):
  print(f'Theta: {theta.deg} degrees')

In [ ]:
showdeg(Angle(60*u.degree))

In [ ]:
#specify the angle in milliarcseconds:
showdeg(Angle(3600*1000*u.mas))

Lots lots lots more at [Units Tutorial](https://docs.astropy.org/en/stable/units/).  Especially see the section [Using Astropy.units](https://docs.astropy.org/en/stable/units/#using-astropy-units). Also take a look at: [Working with Angles](https://docs.astropy.org/en/stable/coordinates/angles.html)


## Coordinates

[Coordinates Class](https://docs.astropy.org/en/stable/coordinates/)


[SkyCoords](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)

In [33]:
#more astropy libraries
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord, EarthLocation

In [ ]:
pleiades = SkyCoord.from_name('pleiades')
pleiades

In [ ]:
#get the right ascension
pleiades.ra

Notice that by default, RA comes out in degrees, minutes, seconds. To see H:M:S, convert to string with a format parmeter:

In [ ]:
pleiades.ra.to_string(u.hour)

In [ ]:
#galactic coordinates
pleiades.galactic

## Catalog Query

### Simbad

Simbad provides on-line catalog information for celestial objects. See [SIMBAD Astronomical Database - CDS (Strasbourg)](http://simbad.u-strasbg.fr/simbad/). Below, we'll query for catalog entries for seven nearby open clusters

In [38]:
clusters = ['Blanco 1',
 'Collinder 140',
 'Coma Berenices Cluster',
 'Hyades',
 'Pleiades',
 'Praesepe',
 'alpha Per']

In [ ]:
# customize the query so that it returns parallax and proper motion measurements
#Need the Simbad module from Astroquery
from astroquery.simbad import Simbad
from astropy.time import Time

mySimbad = Simbad()
mySimbad.add_votable_fields('parallax', 'pm','velocity','typed_id')
mySimbad.get_votable_fields()

In [ ]:
#from astropy.table import Table, vstack
res_table = vstack([mySimbad.query_object(c) for c in clusters],join_type='exact')

In [ ]:
res_table[['TYPED_ID','RA', 'DEC','PMRA','PMDEC','PLX_VALUE','RVZ_RADVEL']]

In [ ]:
#get coordinates for the results, notice vector operations:
clust_coords = SkyCoord(ra=res_table['RA'], dec=res_table['DEC'], unit=(u.hour, u.deg),
                        distance = Distance(parallax=Quantity(res_table['PLX_VALUE'])),
                        pm_ra_cosdec = res_table['PMRA'],
                        pm_dec = res_table['PMDEC'],
                        radial_velocity = res_table['RVZ_RADVEL'] )

clust_coords

In [ ]:
#convert to galactic coords
clust_coords.galactic

In [ ]:
clust_coords.cartesian

In [ ]:
# plot in equatorial coordinates

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111, projection='mollweide')
pcm = ax.pcolormesh(ra_vec.radian,
           dec_vec.radian,
           image_eq,
           cmap='viridis', norm=colors.LogNorm())
#plot the clusters
for c, n in zip(clust_coords, res_table['TYPED_ID']):
  ax.scatter(c.ra.wrap_at(180*u.degree).radian, c.dec.radian, s=500, marker='*',
             label=n.decode('utf-8'))

ax.grid()
ax.set_title('Neutral Hydrogen Density\nEquatorial Coordinates')
ax.legend(loc='upper left', bbox_to_anchor=[1.05, 1.0])

plt.colorbar(pcm, orientation='horizontal',label='Column Density')

## Read FITS file

Astropy can read and write Flexible Image Transport System (fits) files, both locally and remotely. See [FITS File Handling](https://docs.astropy.org/en/stable/io/fits/).

In this section, we'll open a .psrfits file, a .fits file that follows certain conventions for capturing pulsar data. To do so, we need first to mount a storage device into our Colab server.

### Mount drive with demo fits file

In [48]:
from google.colab import drive, auth
auth.authenticate_user()

In [ ]:
#drive.mount('/content/drive')

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
#Mount the drive
!mkdir fits_data
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 society-amateur-radio-astronomers fits_data

### Pulsar Fits File Exploration

Below, we shall explore the structure of a *.psrfits* file. This type of file is the result of an observation session on the Greenbank Observatory 20M telescope in **pulsar** mode. Observations using the telescope's other modes produce different kinds (formats) of *.fits* files.

FITS = Flexible Image Transort System

*.psrfits* appear to be *.fits* specially tailored to capture pulsar observations.  The general case *.psrfits* is documented here:

[PSR FITS file documentation](https://www.atnf.csiro.au/research/pulsar/psrfits_definition/PsrfitsDocumentation.html)

In [ ]:
from astropy.io import fits
fpath = 'fits_data/B0329+54_pulsar.fits'

pulsar_hdul = fits.open(fpath)
pulsar_hdul.info()

In [ ]:
pulsar_hdul[0].header

In [ ]:
hdr1 = pulsar_hdul[1].header
hdr1

#### Subint Data Table

All the observational data is contained in a data table in `pulsar_hdul[1].data`


### Dr. Frank Ghigo (Greenbank 20M Telescope Expert) Expanation:
>>
the first index (NBITS) is the number of bytes per sample,
usually 2, i.e. 16-bit samples.  You have to put them together
 i.e.,  value = byte0 + 256*byte1. (unless its the other way
around.)   The data values ("byte0" and "byte1" have to be
treated as unsigned integers.)
>>
NPOL is usually 4, representing the 4 stokes parameters,
I,Q,U,V.  Usually use just the first one, index=0, which is
I, the sum of the X and Y channels.  Ignore the others unless
you really want to analyze the polarization properties.
>>
If you use python for example:

  ssdata = hdulist[1].data;
  ddata  = ssdata['DATA']`
>>
the order is reverse of the documentation:
  indexes: [nblocks, npol, nchan, nbytes]
>>
Also look at the DAT_WTS column to get the weights.
For 20-meter data there are always 1024 frequency channels,
and they always range from 1300 to 1800 MHz.
Depending on what filter is chosen when you make the observation,
the weights are set to zero for the part of the spectrum that
is not being observed.  If you select the full spectrum, the
weights are set to zero out the part of the spectrum which is usually
full of RFI.
>>
Also use the DAT_FREQ column to identify which channel is which
frequency.  Note the frequency runs in the opposite direction
to the channel number.
>>
--frank

In [ ]:
#Sampling Frequency and intersample time
ist = hdr1['TBIN']
Fs = 1/ist
print(f'Sampling Frequency: {Fs} samples per second, Intersample Time: {ist} seconds')

In [ ]:
#relevant primary header fields:
for f in ['OBS_MODE','NRCVR','OBSNCHAN','FD_POLN','FD_HAND','SCANLEN']:
    print(f'Field: {f}, Value: {pulsar_hdul[0].header[f]}, Comment: {pulsar_hdul[0].header.comments[f]}')

In [ ]:
#relevant subint header fields:
for f in ['NBITS','NBIN','NCHAN','NPOL','NSBLK','POL_TYPE','TUNIT17']:
  print(f'Field: {f}, Value: {hdr1[f]}, Comment: {hdr1.comments[f]}')

In [ ]:
pulsar_data = pulsar_hdul[1].data
pulsar_data.shape

1373 `Subint` records, each record has the following fields:

In [ ]:
hdr = pulsar_hdul[1].header
for i in range(hdr['TFIELDS']):
  f = i+1
  fname = 'TTYPE'+str(f)
  print(f'Field: {f}, Name: {hdr[fname]}, {hdr.comments[fname]}')

Shape of the `Subint` data table:

In [ ]:
#get shape of data table and element data type of first subint record
pulsar_data[0]['DATA'].shape, pulsar_data[0]['DATA'].dtype

Dimension 0 (leftmost) is `pulsar_hdul[1].header['TSBLK']` which specifies the number of samples on each `subint` record. 32 samples per record in this case.

Dimension 1 is `pulsar_hdul[1].header['NPOL']` which is 4, the number of polarisations dimensions, corresponding respectively to Stokes I, Q U and V parameters.

Dimension 2 is `pulsar_hdul[1].header['NCHAN']`, Value of 1024, which is the  number of channels/sub-bands in this file, the center frequency of each channel can be obtained in `pulsar_hdul[1].data[i]['DAT_FREQ']`. Whether each channel is 'good' (not subject to RFI) or 'bad' (subject to RFI) can be found in `pulsar_hdul[1].data[i]['DAT_WTS']`. 1 = good, 0 = bad.

Dimension 3 (rightmost) are the least and most significant bytes, respectively of a sixteen-bit integer value.

In [ ]:
# stack all the subint data tables into  one long array
obs_data_raw = np.concatenate([r['DATA'] for r in pulsar_data])
obs_data_raw.shape

In [ ]:
#combine the two bytes into an int
obs_data_ints = obs_data_raw[:,:,:,0]+obs_data_raw[:,:,:,1]*256
obs_data_ints.shape

In [ ]:
#first transpose to more useful shape, put time dimension as lowest order dim
obs_data_all = np.transpose(obs_data_ints,[1,2,0])
obs_data_all.shape

In [ ]:
#get just the I row (row 0) of Stokes' IQUV
obs_data = obs_data_all[0]
obs_data.shape

In [71]:
#first row of data table
r0 = pulsar_data[0]

In [72]:
stokes_I = obs_data.mean(axis=0)
#get the DAT_WTS from the first subint table record
msk = r0['DAT_WTS'] == 1 # easier as a boolean vector
stokes_I_masked = obs_data[msk,:].mean(axis=0)

In [ ]:
fig = plt.figure(figsize=(16,4))
fig.suptitle(f'Average Spectral Intensity (Stokes\' I)\nBy Time')
ax = fig.subplots(nrows=1, ncols=2)
ax[0].plot(np.arange(len(stokes_I))/Fs, stokes_I)
ax[0].set_title('Full Spectrum')
ax[0].set_ylabel('Stokes I')
ax[0].set_xlabel('Time (Sec)')
ax[0].grid()

ax[1].plot(np.arange(len(stokes_I_masked))/Fs, stokes_I_masked)
ax[1].set_title('RFI Channels Masked')
ax[1].set_ylabel('Stokes I')
ax[1].set_xlabel('Time (Sec)')
ax[1].grid()

#### Observe the Pulse Period

In [ ]:
#look at the period 125 through 150 seconds

fig = plt.figure(figsize=(16,4))
fig.suptitle(f'Average Spectral Intensity (Stokes\' I) By Time')
ax = fig.subplots(nrows=1, ncols=1)
ax.plot(np.arange(len(stokes_I_masked))/Fs, stokes_I_masked)
ax.set_title('RFI Channels Masked')
ax.set_ylabel('Stokes I')
ax.set_xlabel('Time (Sec)')
ax.grid()
ax.set_xlim(125,150)

In [ ]:
#get the sample numbers for the bounds of the window: 125 through 150 seconds
start_i = int(125*Fs); stop_i= int(150*Fs)

# deal with just the observations in the window
stokes_I_win=stokes_I_masked[start_i:stop_i]

start_i, stop_i

We'll use folding to find the interval.

In [121]:
def fold(obsvec, n):
  n_rows = len(obsvec)//n
  n_samples = n_rows*n

  folded = obsvec[:n_samples].reshape((n_rows,-1))

  return folded.mean(axis=0).max()

In [123]:
#we know a-priori that the period is around 0.73 sec, so let's search btwn 0.65 and 0.75
fold_periods = np.arange(int(0.65*Fs), int(0.75*Fs))
peak_pwr = np.array([fold(stokes_I_win, w) for w in fold_periods])

In [ ]:
plt.plot(fold_periods, peak_pwr)

In [129]:
peak = peak_pwr.argmax()
obs_period = fold_periods[peak]/Fs

In [130]:
# Get the GBO Pulsar Catalog
p = 'http://www.gb.nrao.edu/20m/pulsars_all_GBT.cat.txt'
colnames = ['name','jname','alias','ra','dec','l','b','P','DM','S400','S1400', 'binary']
pulsars = pd.read_csv(p, skiprows = 4, delim_whitespace=True,names=colnames)

# Get the period for B0329+54
pub_period = pulsars.query('name == \'B0329+54\'').P.iloc[0]

In [ ]:
print (f'Observed Period:  {obs_period} seconds')
print (f'Published Period: {pub_period} seconds')

#### Exercise for Reader

Compute FFT and measure the period!

In [ ]:
plt.plot(np.fft.fft(stokes_I_masked[start_i:stop_i])[1:])

### Read Remote FITS file

In [ ]:
#read fits file from Vizier
fits_url = 'https://cdsarc.unistra.fr/viz-bin/nph-Cat/fits.gz?J/A+A/616/A10/tablea1b.dat.gz'
tablea1b = Table.read(fits_url)

In [ ]:
tablea1b[0:5]